# Trained and test set predictions for unaligned mixup

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
%cd /content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/

/content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project


In [ ]:
chichewa_embd = np.load('Data/mixup_embeddings/MT5_mixup_embeddings.npy')

In [ ]:
chichewa_embd.shape

(49427, 256)

In [ ]:
chichewa_embd[0][:10]

array([ 0.05803393, -0.03131715,  0.051116  , -0.05522815, -0.01956199,
       -0.02671846,  0.01582509,  0.08930634, -0.02720215, -0.01108595],
      dtype=float32)

In [ ]:
final_comb_arr = np.copy(chichewa_embd)

In [ ]:
train_df = pd.read_csv('Data/mixup_embeddings/MT5_mixup_labels.csv')

In [ ]:
train_df.head()

,Unnamed: 0,0
0,0,POLITICS
1,1,POLITICS
2,2,HEALTH
3,3,POLITICS
4,4,LAW/ORDER


In [ ]:
final_comb_arr.shape

(49427, 256)

In [ ]:
le = LabelEncoder()
train_df['0'] = le.fit_transform(train_df['0'])

In [ ]:
le.classes_

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [ ]:
Y_train = np_utils.to_categorical(train_df['0'], np.unique(train_df['0']).shape[0])

In [ ]:
def get_model(size):
  print('Building model...')
  model = Sequential()
  model.add(Dense(2048, input_shape=(size,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(20))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  #make one, takes in, double that amount, english and chichewa

In [ ]:
print(Y_train.shape)
print(train_df.shape)

(49427, 20)
(49427, 2)


In [ ]:
Y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [ ]:
test_df = pd.read_csv('Data/Test.csv')

In [ ]:
test_df.head()

,ID,Text
0,ID_ADHEtjTi,Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1,ID_AHfJktdQ,Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2,ID_AUJIHpZr,Anatcheleza: Akundiopseza a gogo wanga Akundi...
3,ID_AUKYBbIM,Ulova wafika posauzana Adatenga digiri ya uph...
4,ID_AZnsVPEi,"Dzombe kukoma, koma Kuyambira makedzana, pant..."


In [ ]:
#Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
#print(train_df.shape)
#print(Y_train.shape)

In [ ]:
#tfidf = TfidfVectorizer()
#tfidf_train = tfidf.fit_transform(train_df.Text).toarray()
#tfidf_test = tfidf.transform(X_test).toarray()
train_dataset = tf.data.Dataset.from_tensor_slices((final_comb_arr, Y_train))

#test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test, y_test))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)
model_2 = get_model(final_comb_arr.shape[1])

model_2.fit(train_dataset,epochs=50, batch_size=32,verbose=1)

Building model...
Epoch 1/50
1545/1545 [==============================] - 9s 6ms/step - loss: 0.9411 - accuracy: 0.8235
Epoch 2/50
1545/1545 [==============================] - 8s 5ms/step - loss: 0.4803 - accuracy: 0.8914
Epoch 3/50
1545/1545 [==============================] - 7s 5ms/step - loss: 0.3382 - accuracy: 0.9165
Epoch 4/50
1545/1545 [==============================] - 7s 5ms/step - loss: 0.2939 - accuracy: 0.9286
Epoch 5/50
1545/1545 [==============================] - 7s 5ms/step - loss: 0.2516 - accuracy: 0.9385
Epoch 6/50
1545/1545 [==============================] - 7s 5ms/step - loss: 0.2274 - accuracy: 0.9450
Epoch 7/50
1545/1545 [==============================] - 8s 5ms/step - loss: 0.1975 - accuracy: 0.9527
Epoch 8/50
1545/1545 [==============================] - 7s 5ms/step - loss: 0.1773 - accuracy: 0.9583
Epoch 9/50
1545/1545 [==============================] - 7s 5ms/step - loss: 0.1624 - accuracy: 0.9614
Epoch 10/50
1545/1545 [==============================] - 7s 5ms/

In [ ]:
chichewa_embd_test = np.load('Data/' + 'test_embeddings_chichewa.npy')

In [ ]:
chichewa_embd_test.shape

(620, 256)

In [ ]:
chichewa_embd_test[0][:10]

array([ 0.00155396,  0.05665419,  0.05805837, -0.04778544, -0.00861554,
        0.03276725,  0.04921997,  0.00280924,  0.04463867,  0.02544041],
      dtype=float32)

In [ ]:
final_comb_arr_test = np.copy(chichewa_embd_test)

In [ ]:
final_comb_arr_test.shape

(620, 256)

In [ ]:
#mt5_test = tfidf.transform(test_df.Text).toarray()
#test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test))

In [ ]:
print(test_df.shape)

(620, 2)


In [ ]:
ans = model_2.predict(final_comb_arr_test)

In [ ]:
print(ans.shape)
ans_arr = np.argmax(ans,axis=1)

(620, 20)


In [ ]:
ans_arr[:2]

array([15, 13])

In [ ]:
le.classes_

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [ ]:
arr_class = list(le.inverse_transform(ans_arr))

In [ ]:
arr_class[:5]

['SOCIAL ISSUES', 'RELIGION', 'SOCIAL ISSUES', 'HEALTH', 'SOCIAL ISSUES']

In [ ]:
test_df_copy = test_df.copy()

In [ ]:
print(test_df_copy)

              ID                                               Text
0    ID_ADHEtjTi   Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1    ID_AHfJktdQ  Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2    ID_AUJIHpZr   Anatcheleza: Akundiopseza a gogo wanga Akundi...
3    ID_AUKYBbIM   Ulova wafika posauzana Adatenga digiri ya uph...
4    ID_AZnsVPEi   Dzombe kukoma, koma Kuyambira makedzana, pant...
..           ...                                                ...
615  ID_zdpOUWyJ  Kanyongolo Wapempha Oyimira Milandu Kuti Atsat...
616  ID_zhnOomuu  Amandimenya\nZikomo gogo,\nNdine mtsikana wa z...
617  ID_zmWHvBJb   Apolisi athotha gulu la MYP Asilikali 56 a gu...
618  ID_zphjdFIb   Mwambo wa ukwati wa Chitonga Mtundu wina uliw...
619  ID_ztdtrNxt   Mwapasa autsa mapiri Pamene pali kusamvana pa...

[620 rows x 2 columns]


In [ ]:
test_df_copy["Labels"] = arr_class

In [ ]:
print(test_df_copy)

              ID                                               Text  \
0    ID_ADHEtjTi   Abambo odzikhweza akuchuluka Kafukufuku wa ap...   
1    ID_AHfJktdQ  Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...   
2    ID_AUJIHpZr   Anatcheleza: Akundiopseza a gogo wanga Akundi...   
3    ID_AUKYBbIM   Ulova wafika posauzana Adatenga digiri ya uph...   
4    ID_AZnsVPEi   Dzombe kukoma, koma Kuyambira makedzana, pant...   
..           ...                                                ...   
615  ID_zdpOUWyJ  Kanyongolo Wapempha Oyimira Milandu Kuti Atsat...   
616  ID_zhnOomuu  Amandimenya\nZikomo gogo,\nNdine mtsikana wa z...   
617  ID_zmWHvBJb   Apolisi athotha gulu la MYP Asilikali 56 a gu...   
618  ID_zphjdFIb   Mwambo wa ukwati wa Chitonga Mtundu wina uliw...   
619  ID_ztdtrNxt   Mwapasa autsa mapiri Pamene pali kusamvana pa...   

            Labels  
0    SOCIAL ISSUES  
1         RELIGION  
2    SOCIAL ISSUES  
3           HEALTH  
4    SOCIAL ISSUES  
..             ...  


In [ ]:
subm = pd.read_csv('Data/SampleSubmission.csv')

In [ ]:
subm.head()

,ID,Label
0,ID_sQaPRMWO,0
1,ID_TanclvfR,0
2,ID_CNbveyvk,0
3,ID_MclKMhyP,0
4,ID_rNrmXOGD,0


In [ ]:
subm.drop(columns=['Label'],inplace=True)

In [ ]:
n_df = pd.merge(subm,test_df_copy, left_on='ID', right_on='ID')

In [ ]:
n_df.head()

,ID,Text,Labels
0,ID_sQaPRMWO,Two Sentenced to 5 Years for Armed Robbery Man...,LAW/ORDER
1,ID_TanclvfR,Papa Wakhazikitsa Bungwe Lolimbikitsa Umodzi W...,SOCIAL
2,ID_CNbveyvk,Amalawi a nsomba ali kakasi ku Botswana Ali m...,ECONOMY
3,ID_MclKMhyP,Mtumiki wa Mulungu adalosera Onse okhulupirir...,SOCIAL ISSUES
4,ID_rNrmXOGD,Kubwera kwa 1 000 Kwacha kukusonyeza sizikuye...,ECONOMY


In [ ]:
n_df.rename(columns={"Labels": "Label"},inplace=True)

In [ ]:
n_df.head()

,ID,Text,Label
0,ID_sQaPRMWO,Two Sentenced to 5 Years for Armed Robbery Man...,LAW/ORDER
1,ID_TanclvfR,Papa Wakhazikitsa Bungwe Lolimbikitsa Umodzi W...,SOCIAL
2,ID_CNbveyvk,Amalawi a nsomba ali kakasi ku Botswana Ali m...,ECONOMY
3,ID_MclKMhyP,Mtumiki wa Mulungu adalosera Onse okhulupirir...,SOCIAL ISSUES
4,ID_rNrmXOGD,Kubwera kwa 1 000 Kwacha kukusonyeza sizikuye...,ECONOMY


In [ ]:
n_df.to_csv('Data/MLP_MT5_mixup_50_Submission.csv',columns=['ID','Label'],index=False)